### Interfacing with Other Codes

An important thing to point out about computational material science is that there are many ways to represent each individual atomic system. Force calculators like LAMMPS and ShengBTE occasionally transform the coordinates of your input files while calculating FC's to match their internal convention. Please ensure all coordinate systems are consistent across your simulation.
This especially applies when you calculate harmonic force constants at a high level of theory (e.g. using DFPT for second-order FC's) but use a faster method with different code for third-order FC's.
An example would be if you used second-order FC's from Quantum Espresso, and then use ShengBTE's thirdorder package to calculate third-order FC's by finite difference. The thirdorder package will silently transform unitcell's with negative coordinate when possible.


#### LAMMPS library setup

In order to use LAMMPS with ASE, it needs to be compiled a lib
```bash
cd path/to/lammps/src
make yes-manybody
make yes-molecule
...
make mpi mode=shlib
```
Next, activate your environment
```
conda activate kaldo
```
or if you used `virtualenv`
```
source ~/kaldo/bin/activate
```
and finally install `lammpslib` using
```
make install-python
```
Update 07/2020: It you are having any issue with the July 2020 version of `lammpslib`, try using the October 2019 version.

#### LAMMPS executable setup

You can use LAMMPS executable directly, as illustrated in the example folder.
For that purpose, you will need the  [LAMMPS user-phonon package by Charles Sievers](https://lammps.sandia.gov/doc/Packages_details.html#pkg-user-phonon) to calculate the dynamical matrix and the third order interatomic force constants.

#### Quantum Espresso setup

In order to use QE with ASE, Set the environment variable:
```bash
export ASE_ESPRESSO_COMMAND="mpirun -np 4 /path/to/pw.x -in PREFIX.pwi > PREFIX.pwo"
```
Update 01/2021: It you are having any issue with ASE version 3.20+, try using ASE version 3.19.1.


See the [ASE documentation](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html) for more info.


### Output Storage

#### Default Storage

When using the storage `formatted`, kALDo stores the following dense tensor as formatted, human readable, files:

- `frequency` $(n_{kpoints}, n_{modes})$. mode changes first k changes after
- `velocity_alpha` $(n_{kpoints}, n_{modes})$
- `physical_mode_<min_freq>_<max_freq>_<is_nw>` $(n_{kpoints}, n_{modes})$

- `<temperature>/<statistics>/heat_capacity` $(n_{kpoints}, n_{modes})$
- `<temperature>/<statistics>/population` $(n_{kpoints}, n_{modes})$

- `<temperature>/<statistics>/<method>/<third_bandwidth>/<diffusivity_bandwidth>/conductivity_<alpha>_<beta>` $(n_{kpoints}, n_{modes})$ where the `<third_bandwidth>/<diffusivity_bandwidth>` folder is created only if those values are defined
- `<temperature>/<statistics>/<diffusivity_bandwidth>/diffusivity` $(n_{kpoints}, n_{modes})$
- `<temperature>/<statistics>/<third_bandwidth>/<method>/mean_free_path` $(n_{kpoints}, n_{modes})$
- `<temperature>/<statistics>/<third_bandwidth>/lifetime` $(n_{kpoints}, n_{modes})$
- `<temperature>/<statistics>/<third_bandwidth>/bandwidth` $(n_{kpoints}, n_{modes})$
- `<temperature>/<statistics>/<third_bandwidth>/phase_space` $(n_{kpoints}, n_{modes})$
- `<diffusivity_bandwidth>flux_dense` $(n_{kpoints}, n_{modes}, n_{kpoints}, n_{modes})$, when `diffusivity_threshold` is not specified.
- `<diffusivity_bandwidth>/<diffusivity_threshold>/flux_sparse` $(n_{kpoints}, n_{modes}, n_{kpoints}, n_{modes})$. Sparse only when  `diffusivity_threshold` is specified.


The folder structure depends on the input parameters to the Phonon Object
and in parenthesis is the shape of the tensor. All of the above observables are stored in a dense format, 
except for `flux_alpha` which is stored as formatted file in a `index value` format.

The following tensors are stored in raw binary format and help saving time when performing different simulations on the same sample.

- `_eigensystem (eigenvalues and eigenvectors)`
- `_dynmat_derivatives`
- `<temperature>/<statistics>/<diffusivity_bandwidth>/_generalized_diffusivity` 
- `<temperature>/<statistics>/<third_bandwidth>/_ps_and_gamma_tensor`
- `<temperature>/<statistics>/<third_bandwidth>/_ps_and_gamma`, when only RTA conductivity is required

#### Alternative Storage

Other storage options available are `numpy` and `hdf5` where all the files are saved as one of those formats.
Finally the `memory` option doesn't store any permanent files.

| Measurement             | Units                    |
|-------------------------|--------------------------|
| Distances               | $A$                      |
| Masses                  | $g/N_A$                  |
| Frequencies             | $THz$                    |
| Potential Derivatives   | $eV/(A^2)$ or $eV/(A^3)$ |
| Conductivity [$\kappa$] | $\frac{W}{m\ K}$         |